In [ ]:
import tifffile
import numpy as np
import cupy as cp
import os

DATA_PATH = r'E:\inbox\251015_ACUTEVIS16_mov'

In [5]:
# parse the directory to find the raw video files

def find_raw_file(directory):
    for file in os.listdir(directory):
        if file.endswith('001.raw'):
            return os.path.join(directory, file)
    return None

raw_file_path = find_raw_file(DATA_PATH)

print(f"Raw file path: {raw_file_path}")

Raw file path: E:\inbox\251015_ACUTEVIS16_mov\Image_001_001.raw


In [6]:
# convert raw file to numpy array

raw_as_arr = np.fromfile(raw_file_path, dtype='<u2')

video_array = raw_as_arr.reshape((-1, 512, 512), order='C')

In [8]:
print(raw_as_arr.shape)
print(raw_as_arr.dtype)
print(video_array.shape)

(4771020800,)
uint16
(18200, 512, 512)


In [9]:
# identify raw files with shape larger than (9100, 512, 512)
if video_array.shape[0] > 9100:
    print("The raw file has more than 9100 frames.")
    

The raw file has more than 9100 frames.


In [ ]:
# save as OME-TIFF
tifffile.imwrite(r'E:\inbox\251015_ACUTEVIS14_mov\251015_sub-ACUTEVIS14_ses-02_task-movies_low.tiff', video_array)

In [20]:
# set up suite2p ops
import numpy as np
from suite2p import run_s2p
from suite2p import default_ops

ops = default_ops()

# main settings
ops['tau'] = 2.0 # gcamp8s
ops['fs'] = 15

# IO
ops['save_path0'] = r"E:\Projects\ACUTEVIS\processed"
ops['save_folder'] = '251015_sub-ACUTEVIS14_ses-02_task-movies_low_suite2p'

# output settings
# ops['save_NWB'] = True
ops['combined'] = False

# registration
ops['do_registration']  = True
ops['maxregshift'] = 0.01
ops['keep_movie_raw'] = True

# nonrigid
ops['nonrigid'] = False

# functional detect
ops['denoise'] = True
ops['spatial_scale'] = 1
ops['threshold_scaling'] = 1.5
ops['max_overlap'] = 0.5

# neuropil extraction
ops['inner_neuropil_radius'] = 3
ops['min_neuropil_pixels'] = 300

# spike deconvolution
ops['spikedetect'] = False

np.save(r"E:\Projects\ACUTEVIS\processed\gcamp8s_ops.npy", ops)

In [ ]:
# run suite2p
db = []

db.append({'data_path': [r"E:\Projects\ACUTEVIS\processed"]})

for dbi in db:
    opsEnd = run_s2p(ops = ops, db = dbi)

In [ ]:
ops = np.load(r"E:\Projects\ACUTEVIS\processed\gcamp8s_ops.npy", allow_pickle=True).item()

print(ops)